# Extração e seleção de características

As características que são extraídas dos dados dependem de sua natureza. Os dados EMG são sinais elétricos coletados dentro de um período de tempo, portanto são dados no domínio do tempo. As características deste domínio são extraídas dele. Também é possível transformar os dados EMG para o domínio da frequência e extrair características neste domínio. Existem diversas características que podem ser extraídas de cada domínio, entretando nem todas elas serão relevantes. Cada problema se beneficia de características ou combinações delas. Portanto, é preciso que haja uma seleção de características para encontrar a combinação de características que trará melhor resultado na posterior classificação dos dados.

## Reutilizando os passos anteriores

É necessário carregar os dados pré-processados, para dar início à extração de características. No jupyter notebook podemos utilizar o namespace completo de outro notebook:

In [1]:
%run preprocessing.ipynb

(20000, 2)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
 (60, 2, 20000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 2, 41, 1024) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 2, 41, 513) - (classes+ensaios, canais, janelas, linhas)


Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em EMG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) \\$
    $ sgn(x) = \begin{cases} 1 & \text{if } x \geq threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR-E)

    > $ \frac{1}{N}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N-1}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}[sgn(x_i x_{i+1}) \cap |x_i - x_{i+1}| \geq threshold] \\$
    $ sgn(x) = \begin{cases} 1 & \text{if } x \geq threshold \\ 0 & \text{otherwise} \end{cases} $

### Domínio da frequência

1. Auto Regressive (AR)

    > $ - \sum_{j=1}^{\rho}\alpha_j x_{j-1} + w_n $

2. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

3. Mean Frequency (FMN)

    > $ \sum_{j=1}^{M}f_j PSD_j \Big{ / } \sum_{j=1}^{M}PSD_j $

4. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{i=1}^{M}A_j $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Desafio 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

### Extraindo características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características VAR & RMS (domínio do tempo) e FDM & MMDF (domínio da frequência).

In [35]:
from math import prod

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2


# funções de extração de características

def wamp(x, th):
    res = np.abs(np.diff(x))
    return np.sum(res >= th, axis=-1)

def wl(x):
    res = np.abs(np.diff(x))
    return np.sum(res, axis=-1)

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape[:-1]) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape[:-1]) - 1))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2


array([[[0.13518603, 0.12627554, 0.12716645, ..., 0.14904531,
         0.16904925, 0.15575914],
        [0.02961058, 0.03894236, 0.03939476, ..., 0.06593272,
         0.05775868, 0.05325348]],

       [[0.12635894, 0.13503316, 0.11654829, ..., 0.1405752 ,
         0.13461701, 0.12820154],
        [0.05189286, 0.06410115, 0.06898964, ..., 0.05654533,
         0.05512511, 0.05536497]],

       [[0.09056405, 0.10769047, 0.13284009, ..., 0.13566713,
         0.13177427, 0.14644082],
        [0.02159892, 0.03274034, 0.04119516, ..., 0.0697371 ,
         0.05746156, 0.05891834]],

       ...,

       [[0.17720903, 0.19594147, 0.19201798, ..., 0.21355976,
         0.19527744, 0.25778011],
        [0.09360722, 0.0972943 , 0.10355951, ..., 0.09820558,
         0.09925254, 0.08236675]],

       [[0.1013753 , 0.11747973, 0.16044454, ..., 0.17052863,
         0.17341335, 0.17181153],
        [0.06569758, 0.08767562, 0.12601169, ..., 0.0833443 ,
         0.10011848, 0.10168334]],

       [[0.081283

In [10]:
def moda(x):
    vals,counts = np.unique(x, return_counts=True)
    mode_value = np.argwhere(counts == np.max(counts))
    print(vals)
    
    return mode_value.flatten().tolist(), vals, counts

In [14]:
from scipy import stats

th = np.median(chunks_time)
avr = np.mean(chunks_time)
# m, vals, counts = moda(chunks_time)

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8951119

# ZC

In [22]:
def getZC(data, th):
    tamanho = len(data)
    somatoria = 0
    
    for i in range(tamanho-1):
        resultado = (data[i] * data[i+1] )
        resultado2 = np.abs(data[i] - data[i+1])
        if(resultado < 0 ) and (resultado2 > th):
            somatoria += 1
        
    return somatoria

In [21]:
def zc(data,threshold):
    f =[]
    x,y,z = data.shape[:3]
    for xx in range(x):
        fx = []
        for yy in range(y):
            fy = []
            for zz in range(z):
                fy.append( getZC(data[xx][yy][zz], threshold ) )
            fx.append(fy)
        f.append(fx)
    return np.array(f)

# FMN

In [34]:
def fj(i, sampleRate, tamanho):
    return i * sampleRate / (2 * tamanho)

def getFMN(data):
    tamanho = len(data)
    somatoria = 0
    
    for i in range(tamanho):
        somatoria += (fj(i, 41, tamanho) * PSD(data) ) / np.sum(PSD(data))
        
    return somatoria

def fmn(data):
    f =[]
    x,y,z = data.shape[:3]
    for xx in range(x):
        fx = []
        for yy in range(y):
            fy = []
            for zz in range(z):
                
                fy.append( getFMN(data[xx][yy][zz]) )
                
            fx.append(fy)
        f.append(fx)
    return np.array(f)


In [44]:
resultado = fmn(chunks_freq)
resultado

array([[[[1.39193338e+01, 1.39289650e+01, 1.39764608e+01, ...,
          2.73560927e-01, 2.73556573e-01, 2.73554457e-01],
         [6.66388475e-03, 6.75862915e-03, 7.04510493e-03, ...,
          5.35735859e-07, 5.35585939e-07, 5.35535970e-07],
         [1.58565869e-03, 1.62858504e-03, 1.76205242e-03, ...,
          5.01014196e-07, 5.00831617e-07, 5.00770753e-07],
         ...,
         [2.92647447e-03, 2.95230904e-03, 3.03037866e-03, ...,
          3.99017023e-07, 3.98743287e-07, 3.98652014e-07],
         [1.00622935e+01, 1.00845172e+01, 1.01130057e+01, ...,
          3.34725362e-01, 3.34718064e-01, 3.34715236e-01],
         [3.46195289e+01, 3.45952183e+01, 3.45224628e+01, ...,
          1.39366152e+00, 1.39361704e+00, 1.39360216e+00]],

        [[2.39992857e+00, 2.39381288e+00, 2.40584715e+00, ...,
          1.31578144e-01, 1.31573699e-01, 1.31574125e-01],
         [4.30901558e-03, 4.34436721e-03, 4.44940062e-03, ...,
          8.77426264e-07, 8.77211462e-07, 8.77139871e-07],
        

# MMNF

In [42]:
def A(w):
    return np.abs(w)

def getMMNF(data):
    tamanho = len(data)
    somatoria = 0
    
    for i in range(tamanho):
        somatoria += (fj(i, 41, tamanho) * A(data) ) / np.sum(A(data))
        
    return somatoria

def mmnf(data):
    f =[]
    x,y,z = data.shape[:3]
    for xx in range(x):
        fx = []
        for yy in range(y):
            fy = []
            for zz in range(z):
                
                fy.append( getMMNF(data[xx][yy][zz]) )
                
            fx.append(fy)
        f.append(fx)
    return np.array(f)


In [43]:
resultado = mmnf(chunks_freq)
resultado

array([[[[1.39193338e+01, 1.39289650e+01, 1.39764608e+01, ...,
          2.73560927e-01, 2.73556573e-01, 2.73554457e-01],
         [6.66388475e-03, 6.75862915e-03, 7.04510493e-03, ...,
          5.35735859e-07, 5.35585939e-07, 5.35535970e-07],
         [1.58565869e-03, 1.62858504e-03, 1.76205242e-03, ...,
          5.01014196e-07, 5.00831617e-07, 5.00770753e-07],
         ...,
         [2.92647447e-03, 2.95230904e-03, 3.03037866e-03, ...,
          3.99017023e-07, 3.98743287e-07, 3.98652014e-07],
         [1.00622935e+01, 1.00845172e+01, 1.01130057e+01, ...,
          3.34725362e-01, 3.34718064e-01, 3.34715236e-01],
         [3.46195289e+01, 3.45952183e+01, 3.45224628e+01, ...,
          1.39366152e+00, 1.39361704e+00, 1.39360216e+00]],

        [[2.39992857e+00, 2.39381288e+00, 2.40584715e+00, ...,
          1.31578144e-01, 1.31573699e-01, 1.31574125e-01],
         [4.30901558e-03, 4.34436721e-03, 4.44940062e-03, ...,
          8.77426264e-07, 8.77211462e-07, 8.77139871e-07],
        

In [46]:
resultado.shape

(60, 2, 41, 513)

**Desafio 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características EMG além das apresentadas, citando as fontes de tais características.


## Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

## Implementação do vetor

In [45]:
final_data = list()
final_data.append(var(chunks_time))
final_data.append(rms(chunks_time))
final_data.append(wamp(chunks_time, th))
final_data.append(wl(chunks_time))
final_data.append(zc(chunks_time,0))

final_data.append(fmd(chunks_freq))
final_data.append(mmdf(chunks_freq))
final_data.append(fmn(chunks_freq))
final_data.append(mmnf(chunks_freq))

final = np.array(final_data)
final.shape

/tmp/ipykernel_6494/3753689469.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  final = np.array(final_data)


ValueError: could not broadcast input array from shape (60,2,41,513) into shape (60,2,41)

É necessário que seja reordenado as dimensões do vetor de características, pois cada característica (de cada canal), deve corresponder à última dimensão do vetor. Por fim, as outras dimensões são concatenadas para o número de amostras.

In [20]:
data = final.transpose(1, 3, 2, 0)
X = data.reshape(data.shape[0]*data.shape[1], data.shape[2]*data.shape[3])
X.shape

(2460, 12)

## Seleção de características

Nesta etapa, são selecionadas as características que mais afetam positivamente no resultado final da classificação. Vamos estudar os métodos de seleção de características nesta [página do projeto sklearn](https://scikit-learn.org/stable/modules/feature_selection.html).

**Desafio 3**: mostrar o resultado para os dados de trabalho, para os seguintes métodos seleção de características:
- VarianceThreshold
- Univariate feature selection
    - escolha o que mais for "interessante": `SelectKBest`, `SelectPercentile` e `GenericUnivariateSelect`
- Recursive feature elimination
- Sequential Feature Selection